In [1]:
import time
import asyncio
import pynvml
import subprocess
import pandas as pd
import numpy as np
from typing import Mapping
from functools import cached_property
from emt.power_groups.power_group import PowerGroup


In [2]:
class PowerIntegrator:
    """
    Integrates the instantaneous power usages (W) over the time-delta between the previous call.
    This performs a definite integral of the instantaneous power, using a high-resolution timer,
    the timer measures the time passed since the last call and integrates the power using the
    trapezoidal rule.
    """

    def __init__(self):
        self._previous_time = time.perf_counter()
        self._previous_power = 0.0
        self._energy = 0

    def __call__(self, current_power):
        """
        Add an instantaneous power value (in watts) for a power zone and calculate the cumulative energy
        consumption in Joules.
        Args:
            power_watt (float): Instantaneous power usage in watts.
        Returns:
            float: Cumulative energy consumption in watt-seconds.
        """
        energy_delta = 0
        current_time = time.perf_counter()
        time_delta = current_time - self._previous_time
        # Calculate the energy consumed during this time interval using the trapezoidal rule
        energy_delta = ((current_power + self._previous_power) / 2.0) * time_delta
        self._energy += energy_delta

        # Update the last time for the next call
        self._previous_time = current_time
        return self._energy


In [3]:
pynvml.nvmlInit()
zones = []
power_integrators = []
for index in range(pynvml.nvmlDeviceGetCount()):
    zone_handle = pynvml.nvmlDeviceGetHandleByIndex(index)
    zones.append(zone_handle)
    power_integrators.append(PowerIntegrator())

In [4]:
pynvml.nvmlDeviceGetCount(), zones, power_integrators

(5,
  <__main__.PowerIntegrator at 0x7fb706c02c60>])

In [6]:
names = [ pynvml.nvmlDeviceGetIndex(zone) for zone in zones]
names

[0, 1, 2, 3, 4]

In [7]:
pynvml.nvmlDeviceGetPowerUsage(zones[0])

9690